In [1]:
# Romain THOMAS

In [2]:
import os
import re
from collections import defaultdict

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import ast

C:\Users\romai\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
path =  "../../data/"
dataset = "cb12/"

raw_path = path + dataset + "raw/" 
interim_path = path + dataset + "interim/"
processed_path = path + dataset + "processed/"

encoded_path = interim_path + "infer/"
encoded_path

aes_path = interim_path + "models/"

prediction_path = path + dataset + "predictions/" 

In [4]:
def list_files(repertoire):
    # Liste tous les fichiers dans le répertoire
    fichiers = [f for f in os.listdir(repertoire) if os.path.isfile(os.path.join(repertoire, f))]

    # Dictionnaire pour stocker les listes de fichiers
    fichiers_par_suffixe = defaultdict(list)

    # Expression régulière pour extraire la partie après le dernier "_"
    pattern = re.compile(r'.*_(.*?)\.(keras|csv)$')

    for fichier in fichiers:
        if fichier.endswith('.csv') or fichier.endswith('.keras'):
            match = pattern.search(fichier)
            if match:
                suffixe = match.group(1)
                fichiers_par_suffixe[suffixe].append(fichier)
        else:
            return fichiers
    return fichiers_par_suffixe

In [5]:
encoded_files = list_files(encoded_path)

for suffixe, liste_fichiers in encoded_files.items():
    if suffixe == "test":
        target_files = liste_fichiers
    if suffixe == "train":
        training_files = liste_fichiers
    print(f"Suffixe: {suffixe}")
    print(liste_fichiers)
    print()


saved_aes = list_files(aes_path)

for suffixe, liste_fichiers in saved_aes.items():
    if suffixe == "encoder":
        saved_encoders = liste_fichiers
    print(f"Suffixe: {suffixe}")
    print(liste_fichiers)
    print()

    
print(" ="*20)
print(target_files)
print(training_files)
print(saved_encoders)

Suffixe: test
['content_ae_encoder_test.csv', 'content_dae_encoder_test.csv', 'content_vae_encoder_test.csv']

Suffixe: train
['content_ae_encoder_train.csv', 'content_dae_encoder_train.csv', 'content_vae_encoder_train.csv']

Suffixe: decoder
['content_ae_decoder.keras', 'content_dae_decoder.keras', 'content_vae_decoder.keras']

Suffixe: encoder
['content_ae_encoder.keras', 'content_dae_encoder.keras', 'content_vae_encoder.keras']

 = = = = = = = = = = = = = = = = = = = =
['content_ae_encoder_test.csv', 'content_dae_encoder_test.csv', 'content_vae_encoder_test.csv']
['content_ae_encoder_train.csv', 'content_dae_encoder_train.csv', 'content_vae_encoder_train.csv']
['content_ae_encoder.keras', 'content_dae_encoder.keras', 'content_vae_encoder.keras']


In [6]:
targets_encoded = pd.read_csv(encoded_path + target_files[0], header=0, sep='\t')
targets_encoded.head()

,session_id,latent_session_vector,input_items,remaining_items
0,592,"0.0009184665,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0...",466171,"811982,1013337,565013,854862,277574"
1,592,"0.0017933721,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0...","466171,811982","1013337,565013,854862,277574"
2,592,"0.001597002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","466171,811982,1013337","565013,854862,277574"
3,592,"0.001597002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","466171,811982,1013337,565013","854862,277574"
4,592,"0.001597002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","466171,811982,1013337,565013,854862",277574


In [7]:
training_encoded = pd.read_csv(encoded_path + training_files[0], header=0, sep='\t')
training_encoded.head()

,session_id,latent_session_vector,items
0,3,"0.0014262507,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0...","809208,136489,617374"
1,7,"0.001597002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","206046,787741,663552"
2,11,"0.001597002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","1083186,516837,507614,754917,686406,1058896,33..."
3,18,"0.001597002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","576958,848187,602298,2121"
4,24,"0.001597002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","586119,785425,97989"


In [8]:
def recommend_topk_jobs_with_overlap_filter(targets_df, training_df, k=60, top_n=20):
    """
    Recommend jobs for each test session by selecting training sessions that share at least
    one job with the test session's input_items, and ranking them by cosine similarity.

    Parameters:
    - targets_df: DataFrame with "latent_session_vector" and "input_items" (list of job IDs)
    - training_df: DataFrame with "latent_session_vector" and "items"
    - k: number of most similar sessions to consider
    - top_n: number of jobs to recommend

    Returns:
    - List of job ID lists (top_n recommended jobs per test session), list of cosine similarity arrays
    """
    import numpy as np
    from sklearn.metrics.pairwise import cosine_similarity
    import ast

    # Parse latent vectors
    Z_train = np.array([convert_string_to_array(x) for x in training_df['latent_session_vector']])
    Z_train = np.vstack(Z_train)

    # Parse training sessions' item lists
    train_sessions_jobs = []
    for jobs in training_df["items"].values:
        if isinstance(jobs, str):
            jobs = ast.literal_eval(jobs)
        if isinstance(jobs, int):
            jobs = [jobs]
        train_sessions_jobs.append(set(int(j) for j in jobs))

    recommendations = []
    all_sims = []

    for _, row in targets_df.iterrows():
        z_target = convert_string_to_array(row["latent_session_vector"])

        # Parse input_items
        input_items = row["input_items"]
        if isinstance(input_items, str):
            input_items = ast.literal_eval(input_items)
        if isinstance(input_items, int):
            input_items = [input_items]
        input_items = set(int(j) for j in input_items)

        if not input_items:
            print(" not input_items "+str(k)+" "+str(top_n)+" "+targets_df)
            recommendations.append([" not input_items "])
            all_sims.append(np.array([]))
            continue

        # Find training sessions with at least one job in common
        candidate_idxs = [i for i, jobs in enumerate(train_sessions_jobs) if input_items & jobs]

        if not candidate_idxs:
            print(" not candidate_idxs "+str(k)+" "+str(top_n)+" "+targets_df)
            recommendations.append([" not candidate_idxs "])
            all_sims.append(np.array([]))
            continue

        Z_candidates = Z_train[candidate_idxs]
        sims = cosine_similarity(z_target.reshape(1, -1), Z_candidates).flatten()
        all_sims.append(sims)

        top_k_idx = np.argsort(sims)[-min(k, len(sims)):][::-1]

        job_scores = {}
        for rel_idx in top_k_idx:
            true_idx = candidate_idxs[rel_idx]
            sim_score = sims[rel_idx]
            for job in train_sessions_jobs[true_idx]:
                if job not in input_items:
                    job_scores[job] = job_scores.get(job, 0) + sim_score

        top_jobs = sorted(job_scores.items(), key=lambda x: x[1], reverse=True)[:top_n]
        
        if len(top_jobs) < top_n:
            # Sessions that DID NOT overlap with input_items
            non_overlap_idxs = [i for i in range(len(train_sessions_jobs)) if i not in candidate_idxs]
            if non_overlap_idxs:
                Z_non_overlap = Z_train[non_overlap_idxs]
                sims_non_overlap = cosine_similarity(z_target.reshape(1, -1), Z_non_overlap).flatten()

                # Get sessions with highest cosine similarity
                top_fallback_idxs = np.argsort(sims_non_overlap)[::-1]

                fallback_scores = {}
                for rel_idx in top_fallback_idxs:
                    true_idx = non_overlap_idxs[rel_idx]
                    sim_score = sims_non_overlap[rel_idx]
                    for job in train_sessions_jobs[true_idx]:
                        if job not in input_items and job not in job_scores:
                            fallback_scores[job] = fallback_scores.get(job, 0) + sim_score
                    # Stop early if we already collected enough jobs
                    if len(fallback_scores) >= (top_n - len(top_jobs)):
                        break

                sorted_fallbacks = sorted(fallback_scores.items(), key=lambda x: x[1], reverse=True)
                needed = top_n - len(top_jobs)
                top_jobs += sorted_fallbacks[:needed]


        recommendations.append([job for job, _ in top_jobs])

    return recommendations #, all_sims


In [9]:
# Convert string representations to numpy arrays
def convert_string_to_array(vec_str):
    return np.array([float(x) for x in vec_str.split(',')])


for t in range(1,21,1):
    print(t)

In [10]:
target_files, training_files

(['content_ae_encoder_test.csv',
  'content_dae_encoder_test.csv',
  'content_vae_encoder_test.csv'],
 ['content_ae_encoder_train.csv',
  'content_dae_encoder_train.csv',
  'content_vae_encoder_train.csv'])

In [11]:
training_encoded['latent_session_vector'][0]

'0.0014262507,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00016330004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00073932816,0.0,0.001786733,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0'

In [12]:
type(training_encoded['latent_session_vector'][0])

str

In [13]:
for e in training_encoded['latent_session_vector'][0]:
    print(type(e))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class

In [14]:
for (target, training) in zip(target_files, training_files):
    targets_encoded = pd.read_csv(encoded_path + target, header=0, sep='\t')
    display(targets_encoded.head())
    training_encoded = pd.read_csv(encoded_path + training, header=0, sep='\t')
    display(training_encoded.head())


,session_id,latent_session_vector,input_items,remaining_items
0,592,"0.0009184665,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0...",466171,"811982,1013337,565013,854862,277574"
1,592,"0.0017933721,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0...","466171,811982","1013337,565013,854862,277574"
2,592,"0.001597002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","466171,811982,1013337","565013,854862,277574"
3,592,"0.001597002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","466171,811982,1013337,565013","854862,277574"
4,592,"0.001597002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","466171,811982,1013337,565013,854862",277574


,session_id,latent_session_vector,items
0,3,"0.0014262507,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0...","809208,136489,617374"
1,7,"0.001597002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","206046,787741,663552"
2,11,"0.001597002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","1083186,516837,507614,754917,686406,1058896,33..."
3,18,"0.001597002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","576958,848187,602298,2121"
4,24,"0.001597002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","586119,785425,97989"


,session_id,latent_session_vector,input_items,remaining_items
0,592,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",466171,"811982,1013337,565013,854862,277574"
1,592,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","466171,811982","1013337,565013,854862,277574"
2,592,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","466171,811982,1013337","565013,854862,277574"
3,592,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","466171,811982,1013337,565013","854862,277574"
4,592,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","466171,811982,1013337,565013,854862",277574


,session_id,latent_session_vector,items
0,3,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","809208,136489,617374"
1,7,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","206046,787741,663552"
2,11,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","1083186,516837,507614,754917,686406,1058896,33..."
3,18,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","576958,848187,602298,2121"
4,24,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","586119,785425,97989"


,session_id,latent_session_vector,input_items,remaining_items
0,592,"0.0017225035,0.053339392,0.1214104,0.020633923...",466171,"811982,1013337,565013,854862,277574"
1,592,"0.07972506,-0.0036245026,0.12704223,-0.0136387...","466171,811982","1013337,565013,854862,277574"
2,592,"0.09760852,-0.05731277,0.17136127,-0.021898398...","466171,811982,1013337","565013,854862,277574"
3,592,"0.123271,-0.10945652,0.18692605,-0.026360432,-...","466171,811982,1013337,565013","854862,277574"
4,592,"0.13334964,0.0020997077,0.18070473,-0.04945600...","466171,811982,1013337,565013,854862",277574


,session_id,latent_session_vector,items
0,3,"0.11607601,-0.033259436,0.10646829,-0.01454434...","809208,136489,617374"
1,7,"0.12995647,-0.023657069,0.10086603,0.017356986...","206046,787741,663552"
2,11,"0.12610531,0.12138065,-0.034069147,-0.03514958...","1083186,516837,507614,754917,686406,1058896,33..."
3,18,"0.10977858,-0.025199465,0.12946016,-0.05619090...","576958,848187,602298,2121"
4,24,"0.18712139,-0.027691,0.18927947,0.019358987,-0...","586119,785425,97989"


os.makedirs(os.path.dirname(prediction_path), exist_ok=True)

for k in range(10,110,10): #range(10,110,10)
    print(" *"*20)
    print(" *"*20)
    print("k ="+str(k))
    print(" ")
    for t in range(20, 21,1): #range(1,21,1)
        print(" -"*20)
        print("k ="+str(k))
        print("t ="+str(t))
        print(" ")
        for (target, training) in zip(target_files, training_files):
            print(encoded_path + target)
            targets_encoded = pd.read_csv(encoded_path + target, header=0, sep='\t')
            #display(targets_encoded.head())
            training_encoded = pd.read_csv(encoded_path + training, header=0, sep='\t')
            print(encoded_path + training)
            #display(training_encoded.head())
            targets_encoded["top_"+str(t)+"_recommended_jobs_k="+str(k)] = recommend_topk_jobs_with_overlap_filter(
                targets_encoded,
                training_encoded,
                k=k,
                top_n=t
            )
            
            
            match = re.search(r'_(.*?)_', target)
            if match:
                encoder = match.group(1)
                #print(encoder)  # Output: ae
            
            filename = prediction_path+encoder+"_top_"+str(t)+"_k="+str(k)+".csv"
            targets_encoded.to_csv(filename, sep='\t', index=False)
            print(" -"*20)
            print(filename+" is saved!")
            display(targets_encoded.head())
            print(" ")

#targets_encoded

os.makedirs(os.path.dirname(prediction_path), exist_ok=True)

for (target, training) in zip(target_files, training_files):
    print(encoded_path + target)
    targets_encoded = pd.read_csv(encoded_path + target, header=0, sep='\t')
    #display(targets_encoded.head())
    training_encoded = pd.read_csv(encoded_path + training, header=0, sep='\t')
    print(encoded_path + training)
    #display(training_encoded.head())
    for k in range(10,110,10): #range(10,110,10)
        print(" *"*20)
        print(" *"*20)
        print("k ="+str(k))
        print(" ")
        for t in range(20, 21,1): #range(1,21,1)
            print(" -"*20)
            print("k ="+str(k))
            print("t ="+str(t))
            print(" ")
            targets_encoded["top_"+str(t)+"_recommended_jobs_k="+str(k)] = recommend_topk_jobs_with_overlap_filter(
                targets_encoded,
                training_encoded,
                k=k,
                top_n=t
            )
            
            
            match = re.search(r'_(.*?)_', target)
            if match:
                encoder = match.group(1)
                #print(encoder)  # Output: ae
            
            filename = prediction_path+encoder+"_top_"+str(t)+"_k="+str(k)+".csv"
            targets_encoded.to_csv(filename, sep='\t', index=False)
            print(" -"*20)
            print(filename+" is saved!")
            display(targets_encoded.head())
            print(" ")

#targets_encoded

In [15]:
os.makedirs(os.path.dirname(prediction_path), exist_ok=True)

for (target, training) in zip(target_files, training_files):
    print(encoded_path + target)
    targets_encoded = pd.read_csv(encoded_path + target, header=0, sep='\t')
    #display(targets_encoded.head())
    training_encoded = pd.read_csv(encoded_path + training, header=0, sep='\t')
    print(encoded_path + training)
    #display(training_encoded.head())
    for k in range(10,110,10): #range(10,110,10)
        print(" *"*20)
        print(" *"*20)
        print("k ="+str(k))
        print(" ")
        for t in range(20, 21,1): #range(1,21,1)
            print(" -"*20)
            print("k ="+str(k))
            print("t ="+str(t))
            print(" ")
            targets_encoded["top_"+str(t)+"_recommended_jobs_k="+str(k)] = recommend_topk_jobs_with_overlap_filter(
                targets_encoded,
                training_encoded,
                k=k,
                top_n=t
            )
            
            
    match = re.search(r'_(.*?)_', target)
    if match:
        encoder = match.group(1)
        #print(encoder)  # Output: ae

    filename = prediction_path+encoder+"_top_"+str(t)+".csv"
    targets_encoded.to_csv(filename, sep='\t', index=False)
    print(" -"*20)
    print(filename+" is saved!")
    display(targets_encoded.head())
    print(" ")

#targets_encoded

../../data/cb12/interim/infer/content_ae_encoder_test.csv
../../data/cb12/interim/infer/content_ae_encoder_train.csv
 * * * * * * * * * * * * * * * * * * * *
 * * * * * * * * * * * * * * * * * * * *
k =10
 
 - - - - - - - - - - - - - - - - - - - -
k =10
t =20
 
 * * * * * * * * * * * * * * * * * * * *
 * * * * * * * * * * * * * * * * * * * *
k =20
 
 - - - - - - - - - - - - - - - - - - - -
k =20
t =20
 
 * * * * * * * * * * * * * * * * * * * *
 * * * * * * * * * * * * * * * * * * * *
k =30
 
 - - - - - - - - - - - - - - - - - - - -
k =30
t =20
 
 * * * * * * * * * * * * * * * * * * * *
 * * * * * * * * * * * * * * * * * * * *
k =40
 
 - - - - - - - - - - - - - - - - - - - -
k =40
t =20
 
 * * * * * * * * * * * * * * * * * * * *
 * * * * * * * * * * * * * * * * * * * *
k =50
 
 - - - - - - - - - - - - - - - - - - - -
k =50
t =20
 
 * * * * * * * * * * * * * * * * * * * *
 * * * * * * * * * * * * * * * * * * * *
k =60
 
 - - - - - - - - - - - - - - - - - - - -
k =60
t =20
 
 * * * * * * 

,session_id,latent_session_vector,input_items,remaining_items,top_20_recommended_jobs_k=10,top_20_recommended_jobs_k=20,top_20_recommended_jobs_k=30,top_20_recommended_jobs_k=40,top_20_recommended_jobs_k=50,top_20_recommended_jobs_k=60,top_20_recommended_jobs_k=70,top_20_recommended_jobs_k=80,top_20_recommended_jobs_k=90,top_20_recommended_jobs_k=100
0,592,"0.0009184665,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0...",466171,"811982,1013337,565013,854862,277574","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436..."
1,592,"0.0017933721,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0...","466171,811982","1013337,565013,854862,277574","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,..."
2,592,"0.001597002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","466171,811982,1013337","565013,854862,277574","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592..."
3,592,"0.001597002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","466171,811982,1013337,565013","854862,277574","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592..."
4,592,"0.001597002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","466171,811982,1013337,565013,854862",277574,"[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185..."


 
../../data/cb12/interim/infer/content_dae_encoder_test.csv
../../data/cb12/interim/infer/content_dae_encoder_train.csv
 * * * * * * * * * * * * * * * * * * * *
 * * * * * * * * * * * * * * * * * * * *
k =10
 
 - - - - - - - - - - - - - - - - - - - -
k =10
t =20
 
 * * * * * * * * * * * * * * * * * * * *
 * * * * * * * * * * * * * * * * * * * *
k =20
 
 - - - - - - - - - - - - - - - - - - - -
k =20
t =20
 
 * * * * * * * * * * * * * * * * * * * *
 * * * * * * * * * * * * * * * * * * * *
k =30
 
 - - - - - - - - - - - - - - - - - - - -
k =30
t =20
 
 * * * * * * * * * * * * * * * * * * * *
 * * * * * * * * * * * * * * * * * * * *
k =40
 
 - - - - - - - - - - - - - - - - - - - -
k =40
t =20
 
 * * * * * * * * * * * * * * * * * * * *
 * * * * * * * * * * * * * * * * * * * *
k =50
 
 - - - - - - - - - - - - - - - - - - - -
k =50
t =20
 
 * * * * * * * * * * * * * * * * * * * *
 * * * * * * * * * * * * * * * * * * * *
k =60
 
 - - - - - - - - - - - - - - - - - - - -
k =60
t =20
 
 * * * * 

,session_id,latent_session_vector,input_items,remaining_items,top_20_recommended_jobs_k=10,top_20_recommended_jobs_k=20,top_20_recommended_jobs_k=30,top_20_recommended_jobs_k=40,top_20_recommended_jobs_k=50,top_20_recommended_jobs_k=60,top_20_recommended_jobs_k=70,top_20_recommended_jobs_k=80,top_20_recommended_jobs_k=90,top_20_recommended_jobs_k=100
0,592,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",466171,"811982,1013337,565013,854862,277574","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436..."
1,592,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","466171,811982","1013337,565013,854862,277574","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,..."
2,592,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","466171,811982,1013337","565013,854862,277574","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592..."
3,592,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","466171,811982,1013337,565013","854862,277574","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592..."
4,592,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","466171,811982,1013337,565013,854862",277574,"[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185..."


 
../../data/cb12/interim/infer/content_vae_encoder_test.csv
../../data/cb12/interim/infer/content_vae_encoder_train.csv
 * * * * * * * * * * * * * * * * * * * *
 * * * * * * * * * * * * * * * * * * * *
k =10
 
 - - - - - - - - - - - - - - - - - - - -
k =10
t =20
 
 * * * * * * * * * * * * * * * * * * * *
 * * * * * * * * * * * * * * * * * * * *
k =20
 
 - - - - - - - - - - - - - - - - - - - -
k =20
t =20
 
 * * * * * * * * * * * * * * * * * * * *
 * * * * * * * * * * * * * * * * * * * *
k =30
 
 - - - - - - - - - - - - - - - - - - - -
k =30
t =20
 
 * * * * * * * * * * * * * * * * * * * *
 * * * * * * * * * * * * * * * * * * * *
k =40
 
 - - - - - - - - - - - - - - - - - - - -
k =40
t =20
 
 * * * * * * * * * * * * * * * * * * * *
 * * * * * * * * * * * * * * * * * * * *
k =50
 
 - - - - - - - - - - - - - - - - - - - -
k =50
t =20
 
 * * * * * * * * * * * * * * * * * * * *
 * * * * * * * * * * * * * * * * * * * *
k =60
 
 - - - - - - - - - - - - - - - - - - - -
k =60
t =20
 
 * * * * 

,session_id,latent_session_vector,input_items,remaining_items,top_20_recommended_jobs_k=10,top_20_recommended_jobs_k=20,top_20_recommended_jobs_k=30,top_20_recommended_jobs_k=40,top_20_recommended_jobs_k=50,top_20_recommended_jobs_k=60,top_20_recommended_jobs_k=70,top_20_recommended_jobs_k=80,top_20_recommended_jobs_k=90,top_20_recommended_jobs_k=100
0,592,"0.0017225035,0.053339392,0.1214104,0.020633923...",466171,"811982,1013337,565013,854862,277574","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436..."
1,592,"0.07972506,-0.0036245026,0.12704223,-0.0136387...","466171,811982","1013337,565013,854862,277574","[38565, 854862, 75685, 327436, 837744, 473827,...","[38565, 854862, 75685, 327436, 837744, 473827,...","[38565, 854862, 75685, 327436, 837744, 473827,...","[38565, 854862, 75685, 327436, 837744, 473827,...","[38565, 854862, 75685, 327436, 837744, 473827,...","[38565, 854862, 75685, 327436, 837744, 473827,...","[38565, 854862, 75685, 327436, 837744, 473827,...","[38565, 854862, 75685, 327436, 837744, 473827,...","[38565, 854862, 75685, 327436, 837744, 473827,...","[38565, 854862, 75685, 327436, 837744, 473827,..."
2,592,"0.09760852,-0.05731277,0.17136127,-0.021898398...","466171,811982,1013337","565013,854862,277574","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592..."
3,592,"0.123271,-0.10945652,0.18692605,-0.026360432,-...","466171,811982,1013337,565013","854862,277574","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592..."
4,592,"0.13334964,0.0020997077,0.18070473,-0.04945600...","466171,811982,1013337,565013,854862",277574,"[38565, 327436, 837744, 75685, 1106194, 355928...","[38565, 327436, 837744, 75685, 1106194, 355928...","[38565, 327436, 837744, 75685, 1106194, 355928...","[38565, 327436, 837744, 75685, 1106194, 355928...","[38565, 327436, 837744, 75685, 1106194, 355928...","[38565, 327436, 837744, 75685, 1106194, 355928...","[38565, 327436, 837744, 75685, 1106194, 355928...","[38565, 327436, 837744, 75685, 1106194, 355928...","[38565, 327436, 837744, 75685, 1106194, 355928...","[38565, 327436, 837744, 75685, 1106194, 355928..."


In [16]:
encoder

'vae'

In [17]:
for a in ['ae', 'dae', 'vae']:
    temp = pd.read_csv(prediction_path+a+"_top_"+str(t)+".csv", sep='\t')
    display(temp.head())


,session_id,latent_session_vector,input_items,remaining_items,top_20_recommended_jobs_k=10,top_20_recommended_jobs_k=20,top_20_recommended_jobs_k=30,top_20_recommended_jobs_k=40,top_20_recommended_jobs_k=50,top_20_recommended_jobs_k=60,top_20_recommended_jobs_k=70,top_20_recommended_jobs_k=80,top_20_recommended_jobs_k=90,top_20_recommended_jobs_k=100
0,592,"0.0009184665,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0...",466171,"811982,1013337,565013,854862,277574","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436..."
1,592,"0.0017933721,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0...","466171,811982","1013337,565013,854862,277574","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,..."
2,592,"0.001597002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","466171,811982,1013337","565013,854862,277574","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592..."
3,592,"0.001597002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","466171,811982,1013337,565013","854862,277574","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592..."
4,592,"0.001597002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","466171,811982,1013337,565013,854862",277574,"[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185..."


,session_id,latent_session_vector,input_items,remaining_items,top_20_recommended_jobs_k=10,top_20_recommended_jobs_k=20,top_20_recommended_jobs_k=30,top_20_recommended_jobs_k=40,top_20_recommended_jobs_k=50,top_20_recommended_jobs_k=60,top_20_recommended_jobs_k=70,top_20_recommended_jobs_k=80,top_20_recommended_jobs_k=90,top_20_recommended_jobs_k=100
0,592,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",466171,"811982,1013337,565013,854862,277574","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436..."
1,592,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","466171,811982","1013337,565013,854862,277574","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,..."
2,592,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","466171,811982,1013337","565013,854862,277574","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592..."
3,592,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","466171,811982,1013337,565013","854862,277574","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592..."
4,592,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","466171,811982,1013337,565013,854862",277574,"[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185..."


,session_id,latent_session_vector,input_items,remaining_items,top_20_recommended_jobs_k=10,top_20_recommended_jobs_k=20,top_20_recommended_jobs_k=30,top_20_recommended_jobs_k=40,top_20_recommended_jobs_k=50,top_20_recommended_jobs_k=60,top_20_recommended_jobs_k=70,top_20_recommended_jobs_k=80,top_20_recommended_jobs_k=90,top_20_recommended_jobs_k=100
0,592,"0.0017225035,0.053339392,0.1214104,0.020633923...",466171,"811982,1013337,565013,854862,277574","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436..."
1,592,"0.07972506,-0.0036245026,0.12704223,-0.0136387...","466171,811982","1013337,565013,854862,277574","[38565, 854862, 75685, 327436, 837744, 473827,...","[38565, 854862, 75685, 327436, 837744, 473827,...","[38565, 854862, 75685, 327436, 837744, 473827,...","[38565, 854862, 75685, 327436, 837744, 473827,...","[38565, 854862, 75685, 327436, 837744, 473827,...","[38565, 854862, 75685, 327436, 837744, 473827,...","[38565, 854862, 75685, 327436, 837744, 473827,...","[38565, 854862, 75685, 327436, 837744, 473827,...","[38565, 854862, 75685, 327436, 837744, 473827,...","[38565, 854862, 75685, 327436, 837744, 473827,..."
2,592,"0.09760852,-0.05731277,0.17136127,-0.021898398...","466171,811982,1013337","565013,854862,277574","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592..."
3,592,"0.123271,-0.10945652,0.18692605,-0.026360432,-...","466171,811982,1013337,565013","854862,277574","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592..."
4,592,"0.13334964,0.0020997077,0.18070473,-0.04945600...","466171,811982,1013337,565013,854862",277574,"[38565, 327436, 837744, 75685, 1106194, 355928...","[38565, 327436, 837744, 75685, 1106194, 355928...","[38565, 327436, 837744, 75685, 1106194, 355928...","[38565, 327436, 837744, 75685, 1106194, 355928...","[38565, 327436, 837744, 75685, 1106194, 355928...","[38565, 327436, 837744, 75685, 1106194, 355928...","[38565, 327436, 837744, 75685, 1106194, 355928...","[38565, 327436, 837744, 75685, 1106194, 355928...","[38565, 327436, 837744, 75685, 1106194, 355928...","[38565, 327436, 837744, 75685, 1106194, 355928..."


In [18]:
## debuging...

In [19]:
for enc in ['ae', 'dae', 'vae']:
    df = pd.read_csv(prediction_path + f"{enc}_top_{t}.csv", sep='\t')
    df["vector_norm"] = df["latent_session_vector"].apply(lambda x: np.linalg.norm(convert_string_to_array(x)))
    print(f"{enc} - avg vector norm: {df['vector_norm'].mean()}")


ae - avg vector norm: 0.0029486292493352897
dae - avg vector norm: 0.03733190967097284
vae - avg vector norm: 0.9857558477516811


In [20]:
##Checking that the recommended jobs actually are in the training

In [21]:
## list training job IDs
# Method 1: If your column contains actual lists (not strings)
unique_items = pd.unique(
    [item for sublist in training_encoded['items'] 
     for item in sublist 
     if item != '' and item != ','  # Filter out empty items if needed
    ]
)

# Method 2: If your column contains string representations of lists
unique_items = pd.unique(
    [item for sublist in training_encoded['items'].apply(eval)  # Convert string to list
     for item in sublist 
     if item != '' and item != ','
    ]
)

# For numeric items (convert to integers)
unique_items = sorted([int(x) for x in unique_items if str(x).isdigit()])

print(unique_items)

[22, 60, 62, 81, 89, 97, 101, 107, 123, 131, 132, 141, 142, 156, 165, 169, 172, 178, 180, 182, 186, 212, 244, 261, 303, 309, 310, 314, 318, 342, 353, 358, 370, 382, 383, 389, 390, 395, 400, 406, 407, 408, 413, 414, 418, 426, 429, 441, 444, 445, 458, 459, 460, 465, 470, 480, 481, 482, 505, 509, 510, 511, 518, 519, 532, 535, 544, 547, 548, 551, 556, 573, 578, 581, 591, 592, 595, 598, 606, 607, 608, 610, 612, 613, 614, 619, 625, 626, 627, 628, 631, 635, 636, 643, 644, 648, 663, 665, 669, 676, 685, 711, 727, 745, 755, 761, 769, 778, 802, 807, 811, 830, 836, 837, 842, 844, 848, 849, 859, 861, 957, 958, 982, 994, 1008, 1015, 1017, 1032, 1052, 1108, 1116, 1121, 1160, 1165, 1167, 1171, 1176, 1179, 1194, 1209, 1262, 1265, 1267, 1285, 1348, 1382, 1411, 1427, 1465, 1511, 1513, 1548, 1549, 1551, 1561, 1580, 1594, 1631, 1644, 1651, 1655, 1660, 1736, 1739, 1751, 1760, 1763, 1783, 1840, 1894, 1895, 1898, 1899, 1902, 1904, 1921, 1938, 1951, 1968, 1988, 1989, 1997, 2003, 2006, 2012, 2026, 2039, 2100, 2

C:\Users\romai\AppData\Local\Temp\ipykernel_9616\4285127829.py:3: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  unique_items = pd.unique(
C:\Users\romai\AppData\Local\Temp\ipykernel_9616\4285127829.py:11: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  unique_items = pd.unique(


In [22]:
## list top_n job IDs
# Method 1: If your column contains actual lists (not strings)
unique_items = pd.unique(
    [item for sublist in targets_encoded["top_"+str(t)+"_recommended_jobs_k="+str(k)] 
     for item in sublist 
     if item != '' and item != ','  # Filter out empty items if needed
    ]
)


# For numeric items (convert to integers)
unique_items_sorted = sorted([int(x) for x in unique_items if str(x).isdigit()])

print(unique_items_sorted)

[22, 81, 182, 212, 318, 353, 390, 395, 406, 426, 429, 445, 595, 613, 614, 626, 635, 643, 644, 663, 676, 685, 836, 837, 844, 849, 861, 1167, 1176, 1179, 1209, 1382, 1594, 1894, 1898, 1938, 2006, 2039, 2102, 2168, 2262, 2306, 2313, 2340, 2516, 2580, 2637, 2828, 2880, 2882, 3137, 3176, 4943, 5065, 5109, 5844, 6153, 6246, 6312, 6332, 6690, 6838, 6998, 7084, 7230, 7634, 8695, 8851, 8881, 9897, 10399, 10936, 11023, 11033, 11070, 11263, 11330, 11453, 11498, 11511, 11692, 11712, 11816, 11892, 11913, 11914, 13050, 13106, 13313, 13314, 13433, 13497, 13608, 13687, 13745, 13915, 14195, 14212, 14290, 14310, 14528, 14644, 14866, 14873, 14980, 15037, 15245, 15374, 15716, 15843, 16128, 16209, 16227, 16606, 16677, 16761, 16762, 16990, 17095, 17233, 17304, 17361, 17362, 17439, 17915, 21217, 21280, 22424, 22440, 22873, 22879, 22917, 22952, 23077, 23112, 23443, 23520, 23773, 23893, 23914, 24127, 24204, 24321, 24325, 24457, 24798, 25967, 25983, 26032, 26124, 26828, 26829, 26850, 26917, 27024, 27122, 27155,

C:\Users\romai\AppData\Local\Temp\ipykernel_9616\713580477.py:3: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  unique_items = pd.unique(


In [23]:
## list top_n job IDs
# Flatten all lists and get unique values
all_items = [item for sublist in targets_encoded["top_"+str(t)+"_recommended_jobs_k="+str(k)] for item in sublist]
unique_items = list(set(all_items))

# If you need to remove empty strings or commas
unique_items = [x for x in unique_items if x != ',' and x != '']

# If you want them sorted (optional)
unique_items_sorted = sorted(unique_items, key=lambda x: int(x) if str(x).isdigit() else x)

print(unique_items_sorted)

[22, 81, 182, 212, 318, 353, 390, 395, 406, 426, 429, 445, 595, 613, 614, 626, 635, 643, 644, 663, 676, 685, 836, 837, 844, 849, 861, 1167, 1176, 1179, 1209, 1382, 1594, 1894, 1898, 1938, 2006, 2039, 2102, 2168, 2262, 2306, 2313, 2340, 2516, 2580, 2637, 2828, 2880, 2882, 3137, 3176, 4943, 5065, 5109, 5844, 6153, 6246, 6312, 6332, 6690, 6838, 6998, 7084, 7230, 7634, 8695, 8851, 8881, 9897, 10399, 10936, 11023, 11033, 11070, 11263, 11330, 11453, 11498, 11511, 11692, 11712, 11816, 11892, 11913, 11914, 13050, 13106, 13313, 13314, 13433, 13497, 13608, 13687, 13745, 13915, 14195, 14212, 14290, 14310, 14528, 14644, 14866, 14873, 14980, 15037, 15245, 15374, 15716, 15843, 16128, 16209, 16227, 16606, 16677, 16761, 16762, 16990, 17095, 17233, 17304, 17361, 17362, 17439, 17915, 21217, 21280, 22424, 22440, 22873, 22879, 22917, 22952, 23077, 23112, 23443, 23520, 23773, 23893, 23914, 24127, 24204, 24321, 24325, 24457, 24798, 25967, 25983, 26032, 26124, 26828, 26829, 26850, 26917, 27024, 27122, 27155,

In [24]:
## compare job ID wise

count=0
for e in unique_items_sorted:
    if (e in unique_items)==False:
        print("oups")
    else:
        count+=1
        print(count)
print(" ="*20)
print(len(unique_items_sorted))
print(" ="*20)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [25]:
Z_candidates=[[  0.   ,       0.    ,      0.    ,      0.    ,      0.     ,     0.,
   14.15737  ,   0. ,         0.       ,   0.,          0.  ,       16.96261,
   33.370758  ,  0.   ,      21.470749   , 0.  ,        0.  ,        0.,
    0.        ,  0.    ,      0.        ,  0.   ,       0. ,        53.28056,
   35.27906  ,   0.     ,     0.       ,   0.    ,      0.         , 0.,
    0.      ,   12.023392,    0.      ,    0.     ,   101.35298   ,  0.,
    0.     ,     0.       ,   0.     ,    80.58855 ,    0.       ,   0.,
    0.    ,     14.5294485 ,  0.    ,      0.       ,   0.      ,    0.,
    0.   ,       0.      ,    0.   ,       0.        ,  0.     ,     0.,
    0.      ,   14.099397 ,   0.  ,        0.,          0.    ,      0.,
    0.     ,     0.,          0. ,         0. ,         0.   ,       0.,
    0.    ,      0. ,         0.,          0.  ,        0.  ,        0.,
    0.      ,    0.  ,       75.19242   , 27.704939,    0. ,        31.39731,
   21.968111 ,   0.   ,       0.       ,   0.     ,     0.       ,   0.,
    0.     ,     0.    ,     22.3722  ,   28.376535,    0.      ,    0.,
    0.    ,     27.777191,   39.93098   ,  0.     ,     0.     ,     0.,
    0.   ,      36.876152 ,   0.       ,   0.       ]]
z_target=[0.     ,    0.     ,    0.  ,       0.  ,       0.   ,      0.,
 0.      ,   0.,         0.       ,  0.,         0.     ,    3.0664594,
 0.     ,    2.916828,   5.336643,   0. ,        0.61710143, 0.,
 3.0966983,  0.       ,  1.8611753,  0.  ,       0.       ,  4.7774677,
 0.    ,     0.     ,    0.       ,  4.139621,   0.      ,   7.383164,
 0.   ,      6.862176,   0.        , 0.       ,  0.71293044, 0.,
 0.  ,       0.      ,   0.       ,  1.2691522,  0.        , 0.,
 0. ,        8.917228,   0.      ,   0. ,        0.       ,  0.,
 0.        , 0.   ,      0.33343282, 0.  ,       0.      ,   0.,
 0.5706903,  0.    ,     0.     ,    0.   ,      0.     ,    0.,
 0.   ,      0.60199803, 0.    ,     0.    ,     0.    ,     0.,
 0.  ,       0.    ,     0.   ,      0.      ,   0.   ,      0.,
 0. ,        0.     ,    0.  ,       0.    ,     0.  ,       0.,
 9.629211,   0.      ,   1.8680191,  0.     ,    0. ,        0.,
 0.36109778, 5.9455123,  0. ,        0.      ,   0.,         0.,
 6.924221,   0.      ,   0.,         0.       ,  4.5837946,  0.,
 0.     ,    0.      ,   0. ,        0.        ]
"""
z_target.reshape(1, -1)=[[0. ,        0.     ,    0.      ,   0.   ,      0.    ,     0.,
  0. ,        0.        , 0.   ,      0.      ,   0.   ,      3.0664594,
  0.  ,       2.916828 ,  5.336643,   0.    ,     0.61710143, 0.,
  3.0966983,  0.       ,  1.8611753,  0.   ,      0.  ,       4.7774677,
  0.   ,      0.       ,  0.   ,      4.139621,   0.   ,      7.383164,
  0.    ,     6.862176,   0.    ,     0.        , 0.71293044, 0.,
  0.     ,    0.      ,   0.     ,    1.2691522,  0.  ,       0.,
  0.      ,   8.917228 ,  0.      ,   0.       ,  0.   ,      0.,
  0.       ,  0.     ,    0.33343282, 0.      ,   0.    ,     0.,
  0.5706903,  0.   ,      0.  ,       0.     ,    0.     ,    0.,
  0.    ,     0.60199803, 0.   ,      0.    ,     0.      ,   0.,
  0.     ,    0.   ,      0.    ,     0.   ,      0.       ,  0.,
  0.      ,   0.  ,       0.     ,    0.  ,       0.        , 0.,
  9.629211 ,  0.  ,       1.8680191,  0.     ,    0.   ,      0.,
  0.36109778, 5.9455123,  0.  ,       0.    ,     0. ,        0.,
  6.924221 ,  0.     ,    0.   ,      0. ,        4.5837946,  0.,
  0.    ,     0.  ,       0.    ,     0.        ]]
"""

'\nz_target.reshape(1, -1)=[[0. ,        0.     ,    0.      ,   0.   ,      0.    ,     0.,\n  0. ,        0.        , 0.   ,      0.      ,   0.   ,      3.0664594,\n  0.  ,       2.916828 ,  5.336643,   0.    ,     0.61710143, 0.,\n  3.0966983,  0.       ,  1.8611753,  0.   ,      0.  ,       4.7774677,\n  0.   ,      0.       ,  0.   ,      4.139621,   0.   ,      7.383164,\n  0.    ,     6.862176,   0.    ,     0.        , 0.71293044, 0.,\n  0.     ,    0.      ,   0.     ,    1.2691522,  0.  ,       0.,\n  0.      ,   8.917228 ,  0.      ,   0.       ,  0.   ,      0.,\n  0.       ,  0.     ,    0.33343282, 0.      ,   0.    ,     0.,\n  0.5706903,  0.   ,      0.  ,       0.     ,    0.     ,    0.,\n  0.    ,     0.60199803, 0.   ,      0.    ,     0.      ,   0.,\n  0.     ,    0.   ,      0.    ,     0.   ,      0.       ,  0.,\n  0.      ,   0.  ,       0.     ,    0.  ,       0.        , 0.,\n  9.629211 ,  0.  ,       1.8680191,  0.     ,    0.   ,      0.,\n  0.36109778, 5

In [26]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Ensure z_target is a NumPy array
z_target = np.array(z_target)

# Reshape z_target to match expected input dimensions for cosine_similarity
similarity = cosine_similarity(Z_candidates, z_target.reshape(1, -1))

print(similarity)


[[0.24275996]]
